<a href="https://colab.research.google.com/github/oplee21/MITLAB/blob/Master/XGBoost_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파이썬 래퍼 XGBoost

In [2]:
# 파이썬 래퍼

import xgboost as xgb
from xgboost import plot_importance
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')